In [ ]:
!pip install chromadb
!pip install pandas

In [ ]:
import chromadb
import pandas as pd
from tqdm import tqdm

chroma_client = chromadb.Client()
# collection = chroma_client.create_collection(name="my_collection")
collection = chroma_client.get_or_create_collection(name="my_collection")

df = pd.read_csv('./naver-news-summarization-ko/test_10row.csv', encoding='cp949')
                
df.sample(5)


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-KlueNLI-augSTS')

# 데이터 넣기
ids = []
metadatas = []
embeddings = []

# date category press title document link summary

for row in tqdm(df.iterrows()) :
    index = row[0]
    query = row[1].title
    answer = row[1].summary

    metadata = {
        "query": query,
        "answer": answer
    }

    embedding = model.encode(query, normalize_embeddings=True)

    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)

chunk_size = 1024 # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1 # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings) ]

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size

    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    collection.add(embeddings=chunk_embeddings, ids=chunk_ids,metadatas=chunk_metadatas)


# 검색
result = collection.query(
    query_embeddings=model.encode("코오롱 수소 벨류체인 플랫폼", normalize_embeddings=True).tolist(),
    n_results=3
)

print(result)

In [ ]:
!pip install langchain-chroma
!pip install langchain-community
!pip install langchain-text-splitters

In [15]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_text_splitters import CharacterTextSplitter

loader = CSVLoader('./naver-news-summarization-ko/test_10row.csv', encoding='cp949')
documents = loader.load()

# slpit it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "코오롱 수소 벨류체인 플랫폼"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

date: 2022.7.6 16:27
category: economy
press: 데일리안
title: 코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱
document: ‘코오롱 H2 플랫폼’ 밸류체인 플랫폼 발표 계열사간 역량 결집과 동시에 대외 파트너십 구축 6일 동대문 DDP에서 열린 코리아 H2서밋 인베스터데이에 참석한 코오롱인더스트리 장희구 사장 왼쪽에서 네번째 ⓒ코오롱그룹 데일리안 조인영 기자 코오롱그룹이 수소산업의 밸류체인 전반을 고도화하는 플랫폼 구축에 나선다. 코오롱그룹은 6일 동대문 디자인플라자에서 열린 ‘코리아 H2 비즈니스 서밋 2022 인베스터데이’에 참가해 코오롱인더스트리와 코오롱글로벌 코오롱글로텍 코오롱플라스틱 등 그룹이 보유한 첨단기술을 바탕으로 수소 생산부터 운송 저장 발전사업까지 아우르는 코오롱 H2 플랫폼을 구축한다고 밝혔다. 코오롱인더스트리 장희구 사장은 국내외 수소산업 관련 투자자들을 대상으로 ‘코오롱 H2 플랫폼’의 산업적 가치와 의미를 설명하고 그룹의 역량을 집중하겠다고 발표했다. 최근 세계적인 탄소중립 기조에 발맞춰 수소로 대표되는 청정에너지 사업을 그룹의 지속 가능한 미래를 만들 수 있는 핵심 사업으로 성장시키겠다는 전략이다. 코오롱그룹은 수소연료전지를 중심으로 전개해 온 사업 분야를 확대 재편해 청정수소 생산부터 운송과 저장 이를 이용한 전력생산까지 원스톱 서비스가 가능한 플랫폼을 마련해 나간다. 수소 생산분야는 국내 풍력발전 시장을 선도하고 있는 코오롱글로벌과 협업해 진행한다. 풍력발전은 특성상 야간 및 유휴전력이 발생하게 되는데 이를 활용해 물을 전기 분해하는 수전해방식으로 청정수소를 생산할 예정이다. 경주와 태백 풍력단지를 비롯해 현재 추진 중인 완도 해상풍력단지에 이르기까지 수소생산을 위한 중요한 기반 시설로 활용할 계획이다. 코오롱인더스트리의 수분제어장치 및 전해질 분리막 기술 PEM MEA 이 국내에서 선두를 달리고 있고 상용화도 가능해 사업화에 속도를 낼 수 있을 것으로 보고 있다. 코오롱

In [17]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_text_splitters import CharacterTextSplitter

loader = CSVLoader('./naver-news-summarization-ko/test_10row.csv', encoding='cp949')
documents = loader.load()

# slpit it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
# db = Chroma.from_documents(docs, embedding_function)
db = FAISS.from_documents(docs, embedding_function)

# query it
query = "코오롱 수소 벨류체인 플랫폼"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)